In [ ]:
# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, r2_score
from pandas import read_csv

In [ ]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

In [ ]:
df = pd.read_csv("../input/boston-house-prices/housing.csv",header=None, delimiter=r"\s+", names=column_names)

In [ ]:
df.head(10).T

In [ ]:
df.count()

In [ ]:
df = df[~(df['MEDV'] >= 50.0)]
print(np.shape(df))

In [ ]:
fig, axs = plt.subplots(ncols=7, nrows=2, figsize=(20, 10))
index = 0
axs = axs.flatten()
for k,v in df.items():
    sns.distplot(v, ax=axs[index])
    index += 1
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

In [ ]:
tabcorr = df.corr()

In [ ]:
tabcorr

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(abs(tabcorr), cmap="coolwarm")

In [ ]:
sns.clustermap(abs(tabcorr), cmap="coolwarm")

In [ ]:
from scipy.cluster import hierarchy as hc

corr = 1 - df.corr()
corr_condensed = hc.distance.squareform(corr)
link = hc.linkage(corr_condensed, method='ward')
plt.figure(figsize=(12,12))
den = hc.dendrogram(link, labels=df.columns, orientation='left', leaf_font_size=10)

In [ ]:
correlations = tabcorr.MEDV
print(correlations)

In [ ]:
correlations = correlations.drop(['MEDV'],axis=0)

Les corrélations fortement négatives sont aussi significatives que les positives ; on considère donc les valeurs absolues, et on trie par ordre décroissant 

In [ ]:
print(abs(correlations).sort_values(ascending=False))

On voit que la plus forte corrélation concerne LSTAT (le statut inférieur de la population)

# Régression linéaire multiple

In [ ]:
df.columns

In [ ]:
continuous_features = ['RM','DIS','TAX','PTRATIO']
discrete_features = ['CRIM','ZN','INDUS','CHAS','NOX','AGE','B','LSTAT']

On élimine les features discrètes

In [ ]:
df1 = df.drop(discrete_features, axis=1)

In [ ]:
X = df1.drop(['MEDV'], axis=1)
y = df1.MEDV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lm = LinearRegression()
lm.fit(X_train, y_train)            # apprentissage
y_pred = lm.predict(X_test)         # prédiction sur l'ensemble de test

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_test, y_pred)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Prix")
plt.ylabel("Prediction de prix")
plt.title("Prix reels vs predictions")

In [ ]:
sns.distplot(y_test-y_pred)

In [ ]:
print(np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
scoreR2 = r2_score(y_test, y_pred)
print(scoreR2)

In [ ]:
lm.score(X_test,y_test)

# Regression par forêts aléatoires

In [ ]:
X = df.drop(['MEDV'], axis=1)
y = df.MEDV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestRegressor()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)
print(rf.score(X_test,y_test))

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_test, y_rf)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Prix")
plt.ylabel("Prediction de prix")
plt.title("Prix reels vs predictions")

In [ ]:
sns.distplot(y_test-y_rf)

In [ ]:
print(np.sqrt(mean_squared_error(y_test, y_rf)))

In [ ]:
rf.score(X_test,y_test)

# XGBOOST 

In [ ]:
import xgboost as XGB
xgb  = XGB.XGBRegressor()
xgb.fit(X_train, y_train)
y_xgb = xgb.predict(X_test)
print(xgb.score(X_test,y_test))

plt.figure(figsize=(12,12))
plt.scatter(y_test, y_xgb)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Prix")
plt.ylabel("Prediction de prix")
plt.title("Prix reels vs predictions")